## PTSBE accuracy verification: noisy-reference XEB

This notebook validates that the PTSBE (Pre-Trajectory Sampling with Batch Execution) simulation implementation produces the same measurement distribution as the trusted density-matrix (DM) simulator for identical noisy circuits.

Histogram-based metrics like Hellinger fidelity break down at scale: with $D = 2^n$ possible bitstrings and a fixed shot budget $N$, most bins are empty once $n > 20$, so bin-by-bin comparisons are dominated by sampling noise. XEB avoids this by scoring each sample against the reference probability for that single outcome, giving an estimator that converges at $O(1/\sqrt{N})$ regardless of $D$. For a detailed introduction to XEB theory, see [Google's XEB tutorial](https://quantumai.google/cirq/noise/qcvv/xeb_theory).

We adapt Google's [cross-entropy benchmarking (XEB)](https://quantumai.google/cirq/noise/qcvv/xeb_theory) to work with a noisy reference distribution instead of an ideal statevector (as we can no longer rely on the Porter-Thomas distribution when our reference is noisy). The resulting metric, $F_{\text{noisy}}$, directly measures whether PTSBE samples from the same distribution as the DM simulator.

*Scalability note.* Computing $p_{\text{DM}}(x)$ requires the exact diagonal of the $2^n \times 2^n$ density matrix, which limits this approach to moderate qubit counts. For larger simulations where the full density matrix is infeasible, a sample-only metric like [Maximum Mean Discrepancy (MMD)](https://www.kaggle.com/code/onurtunali/maximum-mean-discrepancy) could compare the two sampling distributions directly, without access to exact probabilities. MMD measures the distance between two distributions using only samples from each, making it applicable at any scale where both PTSBE and standard trajectory simulation can produce shots.

### Circuit family

We use XEB-style random circuits following Google's structure ([Arute et al., Nature 2019](https://www.nature.com/articles/s41586-019-1666-5)). Each circuit has $n$ qubits arranged in a linear chain and $m$ depth cycles. One cycle consists of:

- Single-qubit layer: A random gate from $\{\sqrt{X},\, \sqrt{Y},\, \sqrt{W}\}$ applied to each qubit, where $W = (X + Y)/\sqrt{2}$. No gate repeats on the same qubit in consecutive cycles.
- Two-qubit layer: CX (CNOT) gates in an alternating nearest-neighbor tiling (even pairs on even cycles, odd pairs on odd cycles).

This gate set produces distributions that are highly sensitive to per-gate errors, making it a strong probe for correctness. The implementation below uses $R_y(\theta)$ with random $\theta$ as a practical stand-in for the XEB gate set. The validation methodology is independent of the specific single-qubit gates chosen.

### Deriving the metric

The linear XEB fidelity estimator from [Arute et al.](https://www.nature.com/articles/s41586-019-1666-5) ([Supplementary Information, Eq. 17](https://static-content.springer.com/esm/art%3A10.1038%2Fs41586-019-1666-5/MediaObjects/41586_2019_1666_MOESM1_ESM.pdf)) relates the experimentally observed mean of $Dp_s(q) - 1$ to the fidelity $F$ and the sum of squared ideal probabilities:

$$\overline{\langle Dp_s(q) - 1\rangle} = F \left(D\sum_q p_s(q)^2 - 1\right)$$

Solving for $F$ and writing $e_U = \sum_q p_s(q)^2$ and $\hat{m}_U = \frac{1}{N}\sum_i p_s(x_i)$:

$$F = \frac{\hat{m}_U - 1/D}{e_U - 1/D}$$

In the standard setting, $p_s = p_{\text{ideal}}$ and the Porter-Thomas distribution gives $e_U \approx 2/D$, which simplifies the above to the familiar $F = D\langle p_{\text{ideal}}(x_i)\rangle - 1$. We cannot use this simplification because our reference is a noisy mixed state whose diagonal is not Porter-Thomas distributed.

We replace $p_s$ with $p_{\text{DM}}(x) = \langle x | \rho | x \rangle$, the diagonal of the density matrix from the trusted DM simulator. We then compute $e_U = \sum_x p_{\text{DM}}(x)^2$ exactly from the density matrix, with no distributional assumption:

$$Z = \sum_x p_{\text{DM}}(x)^2$$

The resulting metric is:

$$F_{\text{noisy}} = \frac{\hat{m}_U - 1/D}{Z - 1/D} = \frac{\frac{1}{N}\sum_i p_{\text{DM}}(x_i) \;-\; 1/D}{Z \;-\; 1/D} \qquad \text{where } D = 2^n$$

- $F_{\text{noisy}} = 1.0$: the PTSBE samples reproduce the exact noisy distribution ($\hat{m}_U = Z$, i.e. the cross-correlation with the reference equals the reference's self-score).
- $F_{\text{noisy}} = 0.0$: the PTSBE samples are indistinguishable from uniform ($\hat{m}_U = 1/D$).

### Protocol

For a random circuit $C$ drawn from the family above:

1. **Reference**: Run $C$ with noise on `density-matrix-cpu`. Extract the diagonal of the final density matrix to get $p_{\text{DM}}(x)$ for all $x \in \{0,1\}^n$. Compute $Z = \sum_x p_{\text{DM}}(x)^2$.
2. **PTSBE**: Run the same noisy circuit via `cudaq.ptsbe.sample()` to produce $N$ samples $\{x_1, \ldots, x_N\}$.
3. **Score**: Compute $\hat{m}_U = \frac{1}{N}\sum_i p_{\text{DM}}(x_i)$ and $F_{\text{noisy}} = (\hat{m}_U - 1/D) / (Z - 1/D)$.

In [1]:
import os
os.environ["OMP_NUM_THREADS"] = str(os.cpu_count())

import cudaq
import numpy as np
import time

cudaq.set_target("density-matrix-cpu")
cudaq.set_random_seed(42)

### Section A: Single-circuit walkthrough

We walk through the full protocol on one fixed circuit ($n=8$, $m=8$), introducing each utility function as we need it.

**Circuit construction.** `generate_angles` draws random $R_y$ rotation angles. `build_xeb_circuit` assembles cycles of $R_y$ rotations followed by a nearest-neighbor CNOT chain. Pass `with_mz=False` for `get_state()` (no measurement collapse) and `with_mz=True` for `sample()`.

In [2]:
def generate_angles(n, depth, rng):
    """Generate random rotation angles for an XEB circuit."""
    return rng.uniform(0, 2 * np.pi, size=n * depth).tolist()


def build_xeb_circuit(n, depth, angles, with_mz=True):
    """Build a random-rotation + CNOT-ladder circuit via the builder API."""
    kernel = cudaq.make_kernel()
    q = kernel.qalloc(n)
    idx = 0
    for d in range(depth):
        for i in range(n):
            kernel.ry(angles[idx], q[i])
            idx += 1
        for i in range(n - 1):
            kernel.cx(q[i], q[i + 1])
    if with_mz:
        kernel.mz(q)
    return kernel


rng = np.random.default_rng(0)
test_angles = generate_angles(n=3, depth=2, rng=rng)
test_kernel = build_xeb_circuit(3, 2, test_angles)
counts = cudaq.sample(test_kernel, shots_count=1000)
print(f"3-qubit, depth-2 circuit (no noise): {len(counts)} unique bitstrings from 1000 shots")
print(counts)

3-qubit, depth-2 circuit (no noise): 8 unique bitstrings from 1000 shots
{ 000:108 001:14 010:70 011:3 100:23 101:267 110:55 111:460 }



**Noise model.** PTSBE requires unitary-mixture noise channels (each Kraus operator is a scaled Pauli unitary). `build_noise_model` applies symmetric Pauli noise on single-qubit gates ($X/Y/Z$ each at $p_1/3$), two-qubit gates (15-term Pauli at $p_2/15$ each), and bit-flip readout error ($p_{\text{meas}}$).

In [7]:
def build_noise_model(p1=0.001, p2=0.01, p_meas=0.01):
    """Build a Pauli noise model with measurement readout error.

    PTSBE requires unitary-mixture channels. Pauli channels satisfy this:
    each Kraus operator is a Pauli unitary scaled by sqrt(p).
    """
    noise = cudaq.NoiseModel()
    noise.add_all_qubit_channel('ry', cudaq.Pauli1([p1 / 3] * 3))
    noise.add_all_qubit_channel('cx', cudaq.Pauli2([p2 / 15] * 15))
    noise.add_all_qubit_channel('mz', cudaq.BitFlipChannel(p_meas))
    return noise


noise_model = build_noise_model()
noisy_counts = cudaq.sample(test_kernel, noise_model=noise_model, shots_count=1000)
print(f"Same circuit with Pauli noise (p1=0.001, p2=0.01, p_meas=0.01):")
print(f"  {len(noisy_counts)} unique bitstrings from 1000 shots")
print(noisy_counts)

Same circuit with Pauli noise (p1=0.001, p2=0.01, p_meas=0.01):
  8 unique bitstrings from 1000 shots
{ 000:122 001:28 010:80 011:3 100:47 101:239 110:65 111:416 }



**Step 1: DM reference.** We build both kernel variants (with and without measurement) from the same angles up front, ensuring both simulators run the exact same circuit. `get_dm_diagonal` takes the measurement-free kernel, runs it on `density-matrix-cpu` with noise, and extracts the diagonal $p_{\text{DM}}(x) = \langle x|\rho|x\rangle$.

In [10]:
def get_dm_diagonal(kernel, noise_model):
    """Run DM simulation with noise and return the diagonal probabilities."""
    cudaq.set_noise(noise_model)
    state = cudaq.get_state(kernel)
    cudaq.unset_noise()
    dm = np.array(cudaq.StateMemoryView(state))
    return np.real(np.diag(dm))


n, depth = 8, 8
rng = np.random.default_rng(42)
angles = generate_angles(n, depth, rng)
noise_model = build_noise_model()
D = 2 ** n

kernel_no_mz = build_xeb_circuit(n, depth, angles, with_mz=False)
kernel_mz = build_xeb_circuit(n, depth, angles, with_mz=True)

p_dm = get_dm_diagonal(kernel_no_mz, noise_model)
Z = np.sum(p_dm ** 2)
print(f"Circuit: n={n}, depth={depth}")
print(f"D = 2^{n} = {D}")
print(f"Z = sum p_DM(x)^2 = {Z:.6e}")
print(f"Uniform baseline 1/D = {1.0/D:.6e}")
print(f"Z - 1/D = {Z - 1.0/D:.6e}")
print(f"Top 5 probabilities: {sorted(p_dm, reverse=True)[:5]}")

Circuit: n=8, depth=8
D = 2^8 = 256
Z = sum p_DM(x)^2 = 7.216395e-03
Uniform baseline 1/D = 3.906250e-03
Z - 1/D = 3.310145e-03
Top 5 probabilities: [0.0265998062758281, 0.019600520826974492, 0.019084811620052343, 0.017601808750226464, 0.01651198645024806]


**Step 2: PTSBE sampling.** `run_ptsbe` takes the pre-built measurement kernel, creates a probabilistic sampling strategy, and calls `cudaq.ptsbe.sample()` to produce $N$ bitstring samples.

In [ ]:
def run_ptsbe(kernel, noise_model, shots, max_traj=1000, seed=42):
    """Run PTSBE sampling and return the result."""
    strategy = cudaq.ptsbe.ProbabilisticSamplingStrategy(seed=seed)
    return cudaq.ptsbe.sample(
        kernel, noise_model=noise_model, shots_count=shots,
        sampling_strategy=strategy, max_trajectories=max_traj,
    )


N = 100_000
result = run_ptsbe(kernel_mz, noise_model, shots=N)
print(f"PTSBE sampling: {N} shots, {len(result)} unique bitstrings")

**Step 3: Score.** `compute_f_noisy` looks up $p_{\text{DM}}(x_i)$ for each sampled bitstring, computes the mean $\hat{m}_U$, and returns $F_{\text{noisy}} = (\hat{m}_U - 1/D) / (Z - 1/D)$.

In [ ]:
def compute_f_noisy(p_dm, result, n):
    """Compute noisy-reference XEB fidelity F_noisy."""
    D = 2 ** n
    N = result.get_total_shots()
    Z = np.sum(p_dm ** 2)
    e_ptsbe = 0.0
    for bitstring, count in result.items():
        idx = int(bitstring, 2)
        e_ptsbe += count * p_dm[idx]
    e_ptsbe /= N
    f_noisy = (e_ptsbe - 1.0 / D) / (Z - 1.0 / D)
    return f_noisy, e_ptsbe, Z


f_noisy, e_ptsbe, _ = compute_f_noisy(p_dm, result, n)
print(f"PTSBE results (N={N} shots):")
print(f"  E_PTSBE = {e_ptsbe:.6e}")
print(f"  F_noisy = {f_noisy:.4f}")
print(f"  Pass (F_noisy >= 0.95): {'PASS' if f_noisy >= 0.95 else 'FAIL'}")

### Section B: Negative control

Demonstrate that $F_{\text{noisy}}$ has discriminating power. Two conditions:
- **Matched**: PTSBE uses the same noise model as the DM reference.
- **Mismatched**: PTSBE uses $3\times$ the error rates.

The mismatched case should score well below 1.0, proving the metric detects distribution differences.

In [ ]:
n, depth = 8, 8
rng = np.random.default_rng(100)
angles = generate_angles(n, depth, rng)
kernel_no_mz = build_xeb_circuit(n, depth, angles, with_mz=False)
kernel_mz = build_xeb_circuit(n, depth, angles, with_mz=True)

noise_matched = build_noise_model()
noise_mismatched = build_noise_model(p1=0.003, p2=0.03, p_meas=0.03)

p_dm = get_dm_diagonal(kernel_no_mz, noise_matched)
N = 100_000

result_matched = run_ptsbe(kernel_mz, noise_matched, shots=N)
result_mismatched = run_ptsbe(kernel_mz, noise_mismatched, shots=N)

f_matched, _, _ = compute_f_noisy(p_dm, result_matched, n)
f_mismatched, _, _ = compute_f_noisy(p_dm, result_mismatched, n)

print("Negative control (n=8, depth=8)")
print(f"  Matched noise (p1=0.001, p2=0.01):     F_noisy = {f_matched:.4f}")
print(f"  Mismatched noise (p1=0.003, p2=0.03):  F_noisy = {f_mismatched:.4f}")
print(f"  Separation: {f_matched - f_mismatched:.4f}")

### Section C: Width/depth correctness sweep

$F_{\text{noisy}}$ vs depth for each width. All values should cluster near 1.0. This is the primary correctness demonstration.

The CPU density-matrix simulator stores a $2^n \times 2^n$ complex matrix, so practical limits are $n = 10$-$12$. The full test suite (in CI) should use $n$ up to 24 with GPU-backed simulation.

In [ ]:
widths = [4, 6, 8, 10]
depths = [2, 4, 8]
n_instances = 5
N = 50_000

sweep_results = {}
all_f_values = []
rng = np.random.default_rng(42)

for n in widths:
    noise_model = build_noise_model()
    for depth in depths:
        f_values = []
        for inst in range(n_instances):
            angles = generate_angles(n, depth, rng)
            k_no_mz = build_xeb_circuit(n, depth, angles, with_mz=False)
            k_mz = build_xeb_circuit(n, depth, angles, with_mz=True)
            p_dm = get_dm_diagonal(k_no_mz, noise_model)
            result = run_ptsbe(k_mz, noise_model,
                               shots=N, max_traj=500)
            f, _, _ = compute_f_noisy(p_dm, result, n)
            f_values.append(f)
        mean_f = np.mean(f_values)
        std_f = np.std(f_values)
        sweep_results[(n, depth)] = (mean_f, std_f)
        all_f_values.extend(f_values)
        print(f"n={n:2d}, depth={depth:2d}: "
              f"F_noisy = {mean_f:.4f} +/- {std_f:.4f}")

print(f"\nAll (n, depth) pairs pass F_noisy >= 0.95: "
      f"{all(v[0] >= 0.95 for v in sweep_results.values())}")

### Section D: Trajectory convergence

$F_{\text{noisy}}$ vs number of trajectories for a fixed circuit and fixed total shots. As trajectories increase, $F_{\text{noisy}}$ should converge toward 1.0. This shows how many trajectories are needed for a given accuracy.

In [ ]:
n, depth = 8, 8
N = 100_000
rng = np.random.default_rng(200)
angles = generate_angles(n, depth, rng)
kernel_no_mz = build_xeb_circuit(n, depth, angles, with_mz=False)
kernel_mz = build_xeb_circuit(n, depth, angles, with_mz=True)
noise_model = build_noise_model()
p_dm = get_dm_diagonal(kernel_no_mz, noise_model)

trajectory_counts = [10, 25, 50, 100, 250, 500, 1000]

print(f"Trajectory convergence (n={n}, depth={depth}, N={N})")
for mt in trajectory_counts:
    result = run_ptsbe(kernel_mz, noise_model,
                       shots=N, max_traj=mt)
    f, _, _ = compute_f_noisy(p_dm, result, n)
    print(f"  max_trajectories={mt:5d}: F_noisy = {f:.4f}")

### Section E: Shot convergence

$F_{\text{noisy}}$ vs number of shots $N$ for a fixed circuit and fixed trajectories. Shows how many samples are needed for the metric itself to stabilize. Error bars from multiple runs at each $N$.

In [ ]:
n, depth = 8, 8
max_traj = 500
rng = np.random.default_rng(300)
angles = generate_angles(n, depth, rng)
kernel_no_mz = build_xeb_circuit(n, depth, angles, with_mz=False)
kernel_mz = build_xeb_circuit(n, depth, angles, with_mz=True)
noise_model = build_noise_model()
p_dm = get_dm_diagonal(kernel_no_mz, noise_model)

shot_counts = [1_000, 5_000, 10_000, 50_000, 100_000, 500_000]
n_repeats = 3

print(f"Shot convergence (n={n}, depth={depth}, max_trajectories={max_traj})")
for N in shot_counts:
    f_values = []
    for rep in range(n_repeats):
        result = run_ptsbe(kernel_mz, noise_model,
                           shots=N, max_traj=max_traj, seed=400 + rep)
        f, _, _ = compute_f_noisy(p_dm, result, n)
        f_values.append(f)
    mean_f = np.mean(f_values)
    std_f = np.std(f_values)
    print(f"  N={N:>8d}: F_noisy = {mean_f:.4f} +/- {std_f:.4f}")

### Section F: Noise strength sweep

$F_{\text{noisy}}$ vs single-qubit error rate $p_1$ for a fixed circuit. Both simulators use the same noise at each rate. $F_{\text{noisy}}$ should stay near 1.0 across all rates, confirming the metric is not regime-dependent.

In [ ]:
n, depth = 8, 6
N = 100_000
rng = np.random.default_rng(500)
angles = generate_angles(n, depth, rng)
kernel_no_mz = build_xeb_circuit(n, depth, angles, with_mz=False)
kernel_mz = build_xeb_circuit(n, depth, angles, with_mz=True)

p1_values = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05]

print(f"Noise strength sweep (n={n}, depth={depth}, N={N})")
for p1 in p1_values:
    p2 = p1 * 10
    p_meas = p2
    noise_model = build_noise_model(p1=p1, p2=p2, p_meas=p_meas)
    p_dm = get_dm_diagonal(kernel_no_mz, noise_model)
    result = run_ptsbe(kernel_mz, noise_model,
                       shots=N, max_traj=1000)
    f, _, Z = compute_f_noisy(p_dm, result, n)
    print(f"  p1={p1:.3f}: F_noisy = {f:.4f}  (Z={Z:.6e})")

### Section G: Performance comparison

Wall-clock time vs number of shots for a fixed circuit. Two methods: standard DM sampling (`cudaq.sample`) and PTSBE sampling (`cudaq.ptsbe.sample`). PTSBE amortizes the cost of noise simulation across many shots, so the advantage grows with shot count.

In [ ]:
n, depth = 10, 8
rng = np.random.default_rng(600)
angles = generate_angles(n, depth, rng)
noise_model = build_noise_model()
kernel_mz = build_xeb_circuit(n, depth, angles, with_mz=True)

shot_counts = [1_000, 10_000, 100_000, 1_000_000]

print(f"Performance comparison (n={n}, depth={depth})")
print(f"{'Shots':>10s}  {'DM (s)':>10s}  {'PTSBE (s)':>10s}  {'Speedup':>8s}")
for N in shot_counts:
    t0 = time.perf_counter()
    cudaq.sample(kernel_mz, noise_model=noise_model, shots_count=N)
    t_dm = time.perf_counter() - t0

    t0 = time.perf_counter()
    run_ptsbe(kernel_mz, noise_model, shots=N, max_traj=500)
    t_ptsbe = time.perf_counter() - t0

    speedup = t_dm / t_ptsbe if t_ptsbe > 0 else float('inf')
    print(f"{N:>10d}  {t_dm:>10.3f}  {t_ptsbe:>10.3f}  {speedup:>7.1f}x")

### Section H: $F_{\text{noisy}}$ distribution

Summary statistics of $F_{\text{noisy}}$ across all circuit instances from Section C. A correct implementation produces values tightly concentrated near 1.0.

In [ ]:
f_arr = np.array(all_f_values)
print(f"F_noisy distribution across {len(f_arr)} circuit instances")
print(f"  Mean:   {f_arr.mean():.4f}")
print(f"  Std:    {f_arr.std():.4f}")
print(f"  Min:    {f_arr.min():.4f}")
print(f"  Max:    {f_arr.max():.4f}")
print(f"  Fraction >= 0.95: {np.mean(f_arr >= 0.95):.1%}")